In [ ]:
import sys
import os
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload
from datetime import datetime
import pandas as pd
import numpy as np
import xlwings as xw
import unidecode
import warnings
import time
import pyodbc
import asyncio
print(sys.executable)

In [ ]:
d1 = datetime.now().strftime("%Y%m%d")
d0 = datetime.now().replace(day=1).strftime("%Y%m%d")
num_days = int(d1) - int(d0)
dates_list = pd.date_range(datetime.now().replace(day=1), periods=num_days).strftime("%Y%m%d").tolist()

In [ ]:
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'

cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)

In [ ]:
df0 = pd.read_csv('APIDoanhThu.csv', parse_dates=['DateOrder'], dayfirst=True)

In [ ]:
columns = ['Notes', 'CustomerCode', 'CustomerName', 'CustomerInvoiceCode',
       'CustomerInvoice', 'AccountDebit', 'AccountCredit', 'CashPayable',
       'Unit', 'CashNumber', 'DateReceive', 'IDCodeOrder', 'DateOrder',
       'Symbols', 'InvoiceNumber', 'DateInvoice', 'SalesChannel', 'SaleManID',
       'SaleMan', 'DelivererID', 'Deliverer', 'MemberAddID', 'MemberAdd',
       'ReceiverID', 'Receiver', 'OfficeCode', 'Office']

In [ ]:
def get_tasks():
    tasks = []
    for index, date in enumerate(dates_list):
        date = f"'{dates_list[index]}'"
        query = f"EXEC [Api_GetARDoc] @DateGetData={date}"
        tasks.append(pd.read_sql(query, cnxn))
    return tasks

In [ ]:
tasks = []
for index, date in enumerate(dates_list):
    date = f"'{dates_list[index]}'"
    query = f"EXEC [Api_GetARDoc] @DateGetData={date}"
    tasks.append(pd.read_sql(query, cnxn))
    
print(tasks)

In [ ]:
df1 = pd.DataFrame(columns=columns)

In [ ]:
df1 = pd.DataFrame(columns=columns)
for index, date in enumerate(dates_list):
    date = f"'{dates_list[index]}'"
    query = f"EXEC [Api_GetARDoc] @DateGetData={date}"
    df_from_ps = pd.read_sql(query, cnxn)
    df1 = pd.concat([df1, df_from_ps])

In [ ]:
df = pd.concat([df0, df1])
df['SDDH-CN'] =  df['IDCodeOrder']+'.'+df['OfficeCode']

In [ ]:
df.columns

In [ ]:
df3 = pd.read_csv('raw_data_result.csv', low_memory=False)

In [ ]:
df3['MaVungBH'].unique()

In [ ]:
df3['TenKhuVuc'].unique()

In [ ]:
df3.columns

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
table = pd.pivot_table(df3, values=['MaKenhKH2', 'MaVungBH', 'TenKhuVuc'], index=['SDDH-CN'], aggfunc=pd.Series.nunique)

In [ ]:
table[table['MaKenhKH2']>1]

In [ ]:
table[table['TenKhuVuc']>1]

In [ ]:
df3[df3['SDDH-CN'] == 'DH082021-00945.MR0015']

In [ ]:
len(table[table['MaKenhKH2']>1])
df3 = df[['SDDH-CN','']].copy()
df3.shape

In [ ]:
df4 = df3[['SDDH-CN','MaKenhKH2', 'MaVungBH', 'TenKhuVuc']].copy()

In [ ]:
df4.head()

In [ ]:
df4.shape

In [ ]:
df4.drop_duplicates(keep='first', inplace = True)
df4.shape

In [ ]:
df4[df4['SDDH-CN'].duplicated(keep=False)]

In [ ]:
df4.drop_duplicates(subset=['SDDH-CN'], keep='first', inplace = True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df.merge(df4, how = 'left',left_on='SDDH-CN', right_on='SDDH-CN',suffixes=('_left', '_right'), validate="m:1")

In [ ]:
df.MaKenhKH2.fillna(df['SalesChannel'], inplace=True)

In [ ]:
df.MaKenhKH2.unique()

In [ ]:
df.to_csv('api_doanh_thu.csv')

In [ ]:
# Google Sheet connection
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:/data_sale/datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)

In [ ]:
folder_id = '15j8fcCUpEXDrDWzb8pnYSnW8-7M1Inri'
file_name = 'api_doanh_thu.csv'
file_type = 'text/csv'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid  = files['files'][0]['id']
media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid, body=file_body, media_body=media)
updated_file.execute()